In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor, as_completed
from ALLCools.plot import *
import json

## Comments
- this do not work for all regions, significance is biased to large cluster

## rc

In [2]:
from matplotlib import rc

labelsize = 6
linewidth = 0.6
rc('lines', linewidth=linewidth)
rc('axes', labelsize=labelsize, linewidth=linewidth)
rc('xtick', labelsize=labelsize)
rc('ytick', labelsize=labelsize)
rc('xtick.major', width=linewidth)
rc('ytick.major', width=linewidth)
rc('xtick.minor', width=linewidth-0.2)
rc('ytick.minor', width=linewidth-0.2)


## Cluster Summary

In [3]:
cell_tidy_data = pd.read_msgpack('/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
cell_tidy_data.columns

Index(['AllcPath', 'CCC_Rate', 'CG_Rate', 'CG_RateAdj', 'CH_Rate',
       'CH_RateAdj', 'FinalReads', 'InputReads', 'MappedReads', 'Region',
       'index_name', 'uid', 'BamFilteringRate', 'MappingRate', 'Pos96',
       'Plate', 'Col96', 'Row96', 'Col384', 'Row384', 'FACS_Date', 'Slice',
       'MajorRegion', 'SubRegion', 'CellClass', 'l1-umap_0', 'l1-umap_1',
       'l1-tsne_0', 'l1-tsne_1', 'MajorType', 'l2-umap_0', 'l2-umap_1',
       'l2-tsne_0', 'l2-tsne_1', 'SubType', 'l3-umap_0', 'l3-umap_1',
       'l3-tsne_0', 'l3-tsne_1', 'L1CellClass', 'class_tsne_0', 'class_tsne_1',
       'class_umap_0', 'class_umap_1', 'Order', 'RegionName', 'DetailRegion',
       'PotentialOverlap (MMB)', 'Anterior (CCF coords)',
       'Posterior (CCF coords)', 'MajorRegionColor', 'SubRegionColor',
       'DissectionRegionColor'],
      dtype='object')

In [5]:
cell_tidy_data['Replicate'] = cell_tidy_data['Region'] + '-' + cell_tidy_data['FACS_Date'].astype(str)

In [6]:
cell_tidy_data.head()

,AllcPath,CCC_Rate,CG_Rate,CG_RateAdj,CH_Rate,CH_RateAdj,FinalReads,InputReads,MappedReads,Region,...,Order,RegionName,DetailRegion,PotentialOverlap (MMB),Anterior (CCF coords),Posterior (CCF coords),MajorRegionColor,SubRegionColor,DissectionRegionColor,Replicate
10E_M_0,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.008198,0.822633,0.821166,0.041640,0.033718,1626504.0,4407752,2892347.0,10E,...,41,CA-3,"CA1, CA2, CA3, SUB, ProS","PA, HATA",7500,8100,#d62728,#d62728,#FF6347,10E-190625
10E_M_1,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.006019,0.743035,0.741479,0.024127,0.018218,2009998.0,5524084,3657352.0,10E,...,41,CA-3,"CA1, CA2, CA3, SUB, ProS","PA, HATA",7500,8100,#d62728,#d62728,#FF6347,10E-190625
10E_M_10,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.006569,0.750172,0.748520,0.027665,0.021235,1383636.0,3455260,2172987.0,10E,...,41,CA-3,"CA1, CA2, CA3, SUB, ProS","PA, HATA",7500,8100,#d62728,#d62728,#FF6347,10E-190625
10E_M_101,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.006353,0.760898,0.759369,0.026547,0.020323,2474670.0,7245482,4778768.0,10E,...,41,CA-3,"CA1, CA2, CA3, SUB, ProS","PA, HATA",7500,8100,#d62728,#d62728,#FF6347,10E-190625
10E_M_102,/gale/raidix/rdx-4/mapping/10E/CEMBA190625-10E...,0.005409,0.752980,0.751637,0.019497,0.014164,2430290.0,7004754,4609570.0,10E,...,41,CA-3,"CA1, CA2, CA3, SUB, ProS","PA, HATA",7500,8100,#d62728,#d62728,#FF6347,10E-190625


## Palette

In [7]:
region_palette = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/metadata/palette/dissection_region.palette.csv',
                                header=None, index_col=0, squeeze=True).to_dict()
sub_region_palette = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/metadata/palette/sub_region.palette.csv',
                                header=None, index_col=0, squeeze=True).to_dict()
major_region_palette = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/metadata/palette/major_region.palette.csv',
                                header=None, index_col=0, squeeze=True).to_dict()

cell_class_palette = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/metadata/palette/cell_class.palette.csv',
                                header=None, index_col=0, squeeze=True).to_dict()
major_type_palette = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/metadata/palette/major_type.palette.csv',
                                header=None, index_col=0, squeeze=True).to_dict()
sub_type_palette = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/metadata/palette/sub_type.palette.csv',
                                header=None, index_col=0, squeeze=True).to_dict()



In [8]:
type_order = pd.read_csv('./files/MajorType.order_by_dendrogram.txt', index_col=0, header=None).index.tolist()

## Calc Cluster Region Entropy

In [9]:
from scipy.stats import entropy
from random import shuffle
def calc_rep_entropy(df):
    counts = df['Replicate'].value_counts().tolist()
    return entropy(counts)


def batch_entropy(choose_from, number_to_choose, true_counts, n):
        records = []
        for _ in range(n):
            randome_counts = choose_from.groupby('Region').apply(lambda i: i[
                'Replicate'].sample(number_to_choose[i['Region'][0]])).value_counts()
            e = entropy(randome_counts.tolist(), qk=true_counts.reindex(randome_counts.index).tolist())
            records.append(e)
        return records


def get_random_entropys(choose_from, number_to_choose, n=1000, cpu=10):
    total_records = []
    true_counts = choose_from['Replicate'].value_counts()
    chunk_size = n // cpu
    
    with ProcessPoolExecutor(cpu) as executor:
        futures = []
        for _ in range(cpu):
            futures.append(executor.submit(batch_entropy, 
                                           choose_from.copy(), 
                                           number_to_choose.copy(), 
                                           true_counts.copy(),
                                           chunk_size))
        for future in as_completed(futures):
            total_records += future.result()
    return total_records

In [10]:
n_null = 10000
cpu = 40

In [11]:

null_dict = {}
true_dict = {}
for cluster, cluster_df in cell_tidy_data.groupby('SubType'):
    choose_from = cell_tidy_data[cell_tidy_data['Region'].isin(cluster_df['Region'].unique())]

    true_counts = choose_from['Replicate'].value_counts()
    number_to_choose = cluster_df['Region'].value_counts()
    real_counts = cluster_df['Replicate'].value_counts()
    true_entropy = entropy(real_counts.tolist(), 
                           true_counts.reindex(real_counts.index).tolist())
    
    null_entropy = get_random_entropys(choose_from, number_to_choose, n=n_null, cpu=cpu)
    null_dict[cluster] = null_entropy
    true_dict[cluster] = true_entropy
    print(cluster, (pd.Series(null_entropy) > true_entropy).sum() / n_null)

ANP anp-dg 0.944
ANP anp-olf-cnu 0.408
ASC cortex-olf 0.94
ASC mid 0.024
ASC str-hpf 0.0
CA1 Ak5 0.0
CA1 Chrm3 0.976
CA1 Kif26a 0.056
CA1 Lingo2 0.752
CA1 Outlier 0.844
CA1 Ptprg 0.928
CA3 Cadm2 0.0
CA3 Efnb2 0.996
CA3-St18 Epha5 0.58
CA3-St18 Nuak1 0.0
CA3-St18 Tead1 0.0
CGE-Lamp5 Dock5 0.028
CGE-Lamp5 Grid1 0.1
CGE-Lamp5 Grk5 0.012
CGE-Lamp5 Nrxn3 0.712


Process ForkProcess-818:
Process ForkProcess-819:
Process ForkProcess-836:
Process ForkProcess-812:
Process ForkProcess-838:
Process ForkProcess-816:
Process ForkProcess-820:
Process ForkProcess-839:
Process ForkProcess-837:
Process ForkProcess-822:
Process ForkProcess-828:
Process ForkProcess-811:
Process ForkProcess-823:
Process ForkProcess-821:
Process ForkProcess-831:
Process ForkProcess-832:
Process ForkProcess-830:
Process ForkProcess-814:
Process ForkProcess-840:
Process ForkProcess-827:
Process ForkProcess-824:
Process ForkProcess-826:
Process ForkProcess-834:
Process ForkProcess-835:
Process ForkProcess-817:
Process ForkProcess-813:
Process ForkProcess-833:
Process ForkProcess-829:
Process ForkProcess-825:
Process ForkProcess-815:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Trace

  File "/home/hanliu/miniconda3/envs/clustering/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hanliu/miniconda3/envs/clustering/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hanliu/miniconda3/envs/clustering/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hanliu/miniconda3/envs/clustering/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hanliu/miniconda3/envs/clustering/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hanliu/miniconda3/envs/clustering/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hanliu/miniconda3/envs/clustering/lib/python3.7/multiprocessing/proc

  File "/home/hanliu/miniconda3/envs/clustering/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/hanliu/miniconda3/envs/clustering/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/hanliu/miniconda3/envs/clustering/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/hanliu/miniconda3/envs/clustering/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/hanliu/miniconda3/envs/clustering/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/hanliu/miniconda3/envs/clustering/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/hanliu/miniconda3/envs/clustering/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/hanliu/miniconda3/envs/clustering/lib/python3.7/multiprocessing/queues.py", line 93, in ge

KeyboardInterrupt: 

In [ ]:
with open('files/replicate_kld_null.json', 'w') as f:
    json.dump(null_dict, f)
with open('files/replicate_kld_true.json', 'w') as f:
    json.dump(true_dict, f)

In [15]:
cluster_df = cell_tidy_data[cell_tidy_data['SubType'] == 'CA3 Cadm2']
true_counts = cluster_df['Replicate'].value_counts()

In [17]:
true_counts.sort_index()

10E-190625     78
10E-190627    222
10F-190625     37
10F-190627     26
11E-190214     60
11E-190305     33
11F-190214     92
11F-190305     90
8E-190711     331
8E-190716     418
8J-190711     203
8J-190716     178
9H-190212     125
9H-190219     225
9J-190212      54
9J-190219     112
Name: Replicate, dtype: int64